This notebook is based on that written by Chevalier and can be found on [GitHub](https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition). The differences introduce in this notebook is the use of the Keras RNN library which is more modern compared to the LSTM RNN code used in the aforementioned basis for this notebook. 

# Setup

In [1]:
# Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics

import os

In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
] 

# Extract dataset path

In [3]:
DATA_PATH = "data/"
DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)


Dataset is now located at: data/UCI HAR Dataset/


# Dataset Preparation

In [4]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Substract 1 to each output class for friendly 0-based indexing
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)


In [5]:
# Input Data

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training

learning_rate = 0.0025
lambda_loss_amount = 0.0015
epochs = 5000
#epochs = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 1500


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 9) (2947, 1) 0.09913992 0.39567086
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [6]:
print(y_train)

[[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]


## Define RNN

In [7]:
model = keras.Sequential()
# 128 timesteps with 9 features each 
model.add(keras.Input(shape=(128,9)))
model.add(layers.LSTM(n_hidden, activation='relu', return_sequences=True))
model.add(layers.LSTM(n_hidden, activation='relu', return_sequences=False))
# model.add(layers.SimpleRNN(n_hidden, activation='relu', return_sequences=True))
# model.add(layers.SimpleRNN(n_hidden, activation='relu',))
# model.add(layers.SimpleRNN(n_hidden, activation='relu'))
model.add(layers.Dense(n_classes))
model.summary()

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate=learning_rate)
metrics = ["accuracy", "categorical_accuracy"]
model.compile(loss=loss, optimizer=optim, metrics=metrics)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 32)           5376      
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


## Train the net

In [8]:
#print(f"X_train shape: {X_train.shape}, Y_train shape: {y_train.shape}")
#print(f"X_train values: {X_train}")
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/5000
5/5 - 9s - loss: 1.7617 - accuracy: 0.2748 - categorical_accuracy: 1.3602e-04
Epoch 2/5000
5/5 - 5s - loss: 5.6332 - accuracy: 0.3483 - categorical_accuracy: 0.0023
Epoch 3/5000
5/5 - 6s - loss: 64638024.0000 - accuracy: 0.2383 - categorical_accuracy: 0.0556
Epoch 4/5000
5/5 - 7s - loss: 849540808704.0000 - accuracy: 0.2266 - categorical_accuracy: 0.2667
Epoch 5/5000
5/5 - 6s - loss: 436890959872.0000 - accuracy: 0.1595 - categorical_accuracy: 0.5529
Epoch 6/5000
5/5 - 7s - loss: 13163252736.0000 - accuracy: 0.1688 - categorical_accuracy: 4.0805e-04
Epoch 7/5000
5/5 - 6s - loss: 905137280.0000 - accuracy: 0.1676 - categorical_accuracy: 2.7203e-04
Epoch 8/5000
5/5 - 6s - loss: 160446496.0000 - accuracy: 0.1589 - categorical_accuracy: 2.7203e-04
Epoch 9/5000
5/5 - 5s - loss: 44117904.0000 - accuracy: 0.1595 - categorical_accuracy: 0.0000e+00
Epoch 10/5000
5/5 - 5s - loss: 10951359.0000 - accuracy: 0.1631 - categorical_accuracy: 0.0322
Epoch 11/5000
5/5 - 5s - loss: 8246360.5

Epoch 85/5000
5/5 - 9s - loss: 5387806.5000 - accuracy: 0.1598 - categorical_accuracy: 9.5212e-04
Epoch 86/5000
5/5 - 9s - loss: 5365216.0000 - accuracy: 0.1598 - categorical_accuracy: 9.5212e-04
Epoch 87/5000
5/5 - 9s - loss: 5342192.0000 - accuracy: 0.1600 - categorical_accuracy: 0.0011
Epoch 88/5000
5/5 - 9s - loss: 5318547.0000 - accuracy: 0.1598 - categorical_accuracy: 0.0012
Epoch 89/5000
5/5 - 9s - loss: 5295119.0000 - accuracy: 0.1598 - categorical_accuracy: 0.0011
Epoch 90/5000
5/5 - 9s - loss: 5272193.5000 - accuracy: 0.1598 - categorical_accuracy: 0.0012
Epoch 91/5000
5/5 - 8s - loss: 5248891.5000 - accuracy: 0.1598 - categorical_accuracy: 0.0011
Epoch 92/5000
5/5 - 8s - loss: 5225631.0000 - accuracy: 0.1600 - categorical_accuracy: 0.0011
Epoch 93/5000
5/5 - 8s - loss: 5202977.5000 - accuracy: 0.1598 - categorical_accuracy: 0.0011
Epoch 94/5000
5/5 - 7s - loss: 5180508.0000 - accuracy: 0.1598 - categorical_accuracy: 0.0011
Epoch 95/5000
5/5 - 7s - loss: 5158601.5000 - accura

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test, batch_size=batch_size, verbose=2)

In [ ]:
model.save("HAR-RNN-Classifier")